Generative Chatbot
==================
> CSCK507 Group Project: Group A
> 
This is a generative chatbot that uses a seq2seq model.

# Importing Libraries

In [1]:
import requests
import zipfile
import io
import os
import tarfile

# Data Preprocessing

## Importing the dataset

In [2]:
def download_file(url, dir):
    """
    Download file from url
    :param url: url of file
    :param filename: name of file
    :return: None
    """
    r = requests.get(url)
    if url.endswith('.tar.gz'):
        z = tarfile.open(fileobj=io.BytesIO(r.content), mode="r:gz")
        z.extractall(dir)
        z.close()
    elif url.endswith('.zip'):
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall(dir)
    else:
        print('Unknown file type')
    return None

def extract_zip(filename, dir):
    """
    Extract zip file
    :param filename: name of file
    :return: None
    """
    z = zipfile.ZipFile(filename)
    z.extractall(dir)
    return None

In [3]:
datasets = {'WikiQACorpus': 'https://download.microsoft.com/download/E/5/F/E5FCFCEE-7005-4814-853D-DAA7C66507E0/WikiQACorpus.zip',
            'Question_Answer_Dataset_v1.2': 'https://www.cs.cmu.edu/~ark/QA-data/data/Question_Answer_Dataset_v1.2.tar.gz',
            'ubuntu-dialogue': 'data/ubuntu dialogue.zip'}

In [4]:
# Create directory
if not os.path.exists('data'):
    os.makedirs('data')

# Check if data is already downloaded
for dataset, source in datasets.items():
    if os.path.exists('data/' + dataset):
        print(dataset + ' already exists')
    elif dataset == 'ubuntu-dialogue':
        ubuntu = 'data/ubuntu dialogue'
        if os.path.exists(source):
            os.makedirs(ubuntu)
            extract_zip(source, ubuntu)
            os.remove(source)
            print(dataset + ' extracted')
        elif os.path.exists(ubuntu):
            print(dataset + ' already exists')
        else:
            kag = 'https://www.kaggle.com/datasets/rtatman/ubuntu-dialogue-corpus/download?datasetVersionNumber=2'
            print(f'Manually download ubuntu dialogue dataset from {kag} and place in data folder')
    else:
        download_file(source, 'data')
        print(dataset + ' downloaded')

WikiQACorpus already exists
Question_Answer_Dataset_v1.2 already exists
ubuntu-dialogue already exists


## Preparing the dataset